In [1]:
from utils import get_path, group_cols
from datatable import dt, f, as_type

# Descrição

Neste notebook iremos formar uma tabela contendo todos os municípios de residência que estão na base do SIH.

# Tabelas base

## SIH

### Colunas da tabela

In [2]:
cols_mun = [
  'cod_municipio',
  'regiao',
  'uf',
  'cod_uf',
  'nome_uf',
  'municipio',
  'nome_municipio',
  'capital',
  'fronteira',
  'amazonia',
  'macroregiao_saude',
  # 'regiao_saude',
  # 'nome_regiao_saude',
  'microregiao_saude',
  'latitude',
  'longitude',
  'altitude',
  'area',
]

In [3]:
path_sih = get_path('SIH', 'SIH.jay')
df_mun = dt.fread(path_sih)
df_mun = df_mun[:, cols_mun]
df_mun

,cod_municipio,regiao,uf,cod_uf,nome_uf,municipio,nome_municipio,capital,fronteira,amazonia,…,microregiao_saude,latitude,longitude,altitude,area
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,120005,Norte,AC,12,ACRE,Assis Brasil,ASSIS BRASIL,N,S,S,…,12900,−10.941,−69.567,239,2875.91
1,120040,Norte,AC,12,ACRE,Rio Branco,RIO BRANCO,S,S,S,…,12900,−9.975,−67.81,153,9222.58
2,120040,Norte,AC,12,ACRE,Rio Branco,RIO BRANCO,S,S,S,…,12900,−9.975,−67.81,153,9222.58
3,120040,Norte,AC,12,ACRE,Rio Branco,RIO BRANCO,S,S,S,…,12900,−9.975,−67.81,153,9222.58
4,120040,Norte,AC,12,ACRE,Rio Branco,RIO BRANCO,S,S,S,…,12900,−9.975,−67.81,153,9222.58
5,120040,Norte,AC,12,ACRE,Rio Branco,RIO BRANCO,S,S,S,…,12900,−9.975,−67.81,153,9222.58
6,120040,Norte,AC,12,ACRE,Rio Branco,RIO BRANCO,S,S,S,…,12900,−9.975,−67.81,153,9222.58
7,120005,Norte,AC,12,ACRE,Assis Brasil,ASSIS BRASIL,N,S,S,…,12900,−10.941,−69.567,239,2875.91
8,120040,Norte,AC,12,ACRE,Rio Branco,RIO BRANCO,S,S,S,…,12900,−9.975,−67.81,153,9222.58


### Tabela agrupada

In [4]:
df_mun = group_cols(df_mun, cols_mun)
df_mun = df_mun.sort(-f.count)
df_mun

,cod_municipio,regiao,uf,cod_uf,nome_uf,municipio,nome_municipio,capital,fronteira,amazonia,…,latitude,longitude,altitude,area,count
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,355030,Sudeste,SP,35,SAO PAULO,São Paulo,SAO PAULO,S,N,N,…,−23.548,−46.636,760,1522.99,806352
1,330455,Sudeste,RJ,33,RIO DE JANEIRO,Rio de Janeiro,RIO DE JANEIRO,S,N,N,…,−22.903,−43.208,2,1182.3,368216
2,130260,Norte,AM,13,AMAZONAS,Manaus,MANAUS,S,N,S,…,−3.102,−60.025,92,11401.1,297036
3,230440,Nordeste,CE,23,CEARA,Fortaleza,FORTALEZA,S,N,N,…,−3.717,−38.543,21,313.14,203370
4,292740,Nordeste,BA,29,BAHIA,Salvador,SALVADOR,S,N,N,…,−12.971,−38.511,8,706.799,169924
5,150140,Norte,PA,15,PARA,Belém,BELEM,S,N,S,…,−1.456,−48.504,10,1064.92,126918
6,310620,Sudeste,MG,31,MINAS GERAIS,Belo Horizonte,BELO HORIZONTE,S,N,N,…,−19.817,−43.956,858,330.954,125370
7,431490,Sul,RS,43,RIO GRANDE DO SUL,Porto Alegre,PORTO ALEGRE,S,N,N,…,−30.033,−51.23,3,496.827,105321
8,410690,Sul,PR,41,PARANA,Curitiba,CURITIBA,S,N,N,…,−25.428,−49.273,934,434.967,101337


## Índice Socioeconômico

### Colunas da tabela

In [5]:
cols_socioecon = {
  'CÓDIGO': 'codigo',
  'Tipo Localidade': 'tipo',
  'CIR': 'nome_regiao_saude',
  'Grupos de Regiões de Saúde segundo Condições Socioeconômicas e de Saúde - 2016': 'socioecon',
  ...: None
}

In [6]:
path_socioeco = path_socioeco = get_path('INDICADORES', 'indicadores.csv.gzip')
df_socioeco = dt.fread(path_socioeco, columns=cols_socioecon)
df_socioeco = df_socioeco.to_pandas()
df_socioeco

,codigo,tipo,nome_regiao_saude,socioecon
0,15.001,REGSAU,Araguaia,1
1,15.002,REGSAU,Baixo Amazonas,1
2,15.003,REGSAU,Carajás,2
3,15.004,REGSAU,Lago de Tucuruí,1
4,15.013,REGSAU,Marajó I,1
...,...,...,...,...
6003,171.780,MUNIC,Sudeste,1
6004,171.800,MUNIC,Sudeste,1
6005,171.865,MUNIC,Sudeste,1
6006,172.090,MUNIC,Sudeste,1


### Municípios

In [7]:
df_munic = df_socioeco.loc[df_socioeco['tipo'] == 'MUNIC'].copy()
df_munic.drop(columns=['tipo'], inplace=True)
df_munic['codigo'] = df_munic['codigo'].apply(lambda x: (str(x).replace('.', '') + '00')[:6])
df_munic['prefix'] = df_munic['codigo'].apply(lambda x: str(x)[:2])
df_munic = df_munic.rename(columns={'codigo': 'cod_municipio'})
df_munic


,cod_municipio,nome_regiao_saude,socioecon,prefix
13,310600,João Monlevade,3,31
14,313620,João Monlevade,3,31
15,314470,João Monlevade,3,31
16,315570,João Monlevade,3,31
17,316100,João Monlevade,3,31
...,...,...,...,...
6003,171780,Sudeste,1,17
6004,171800,Sudeste,1,17
6005,171865,Sudeste,1,17
6006,172090,Sudeste,1,17


### Regiões de Saúde

In [8]:
df_regsau = df_socioeco[df_socioeco['tipo'] == 'REGSAU'].copy()
df_regsau.drop(columns=['tipo', 'socioecon'], inplace=True)
df_regsau['codigo'] = df_regsau['codigo'].apply(lambda x: (str(x).replace('.', '') + '0')[:5])
df_regsau['prefix'] = df_regsau['codigo'].apply(lambda x: str(x)[:2])
df_regsau = df_regsau.rename(columns={'codigo': 'regiao_saude'})
df_regsau

,regiao_saude,nome_regiao_saude,prefix
0,15001,Araguaia,15
1,15002,Baixo Amazonas,15
2,15003,Carajás,15
3,15004,Lago de Tucuruí,15
4,15013,Marajó I,15
...,...,...,...
5431,17001,Médio Norte Araguaia,17
5432,28005,Nossa Senhora da Glória,28
5433,28006,Nossa Senhora do Socorro,28
5434,28007,Propriá,28


### Merge

In [9]:
df_socioeco = df_munic.merge(df_regsau, how='left', left_on=['prefix', 'nome_regiao_saude'], right_on=['prefix', 'nome_regiao_saude'])
df_socioeco = dt.Frame(df_socioeco)
df_socioeco[:, 'cod_municipio'] = df_socioeco[:, dt.as_type(f.cod_municipio, int)]
df_socioeco

,cod_municipio,nome_regiao_saude,socioecon,prefix,regiao_saude
,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,310600,João Monlevade,3,31,31035
1,313620,João Monlevade,3,31,31035
2,314470,João Monlevade,3,31,31035
3,315570,João Monlevade,3,31,31035
4,316100,João Monlevade,3,31,31035
5,310855,João Pinheiro,2,31,31036
6,313630,João Pinheiro,2,31,31036
7,313753,João Pinheiro,2,31,31036
8,310280,Juiz de Fora Lima Duarte Bom Jardim Min,5,31,31037


# Gerando tabela

## Juntando com outras

## Juntando com Socioeconômico

In [10]:
df_socioeco.key = 'cod_municipio'
df_mun = df_mun[:, :, dt.join(df_socioeco)]
cols_mun = list(df_mun.names)
cols_mun = cols_mun[:-2] + [cols_mun[-1]] + [cols_mun[-2]]
df_mun = df_mun[:, cols_mun]
df_mun

,cod_municipio,regiao,uf,cod_uf,nome_uf,municipio,nome_municipio,capital,fronteira,amazonia,…,count,nome_regiao_saude,socioecon,regiao_saude,prefix
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,355030,Sudeste,SP,35,SAO PAULO,São Paulo,SAO PAULO,S,N,N,…,806352,São Paulo,5,35016,35
1,330455,Sudeste,RJ,33,RIO DE JANEIRO,Rio de Janeiro,RIO DE JANEIRO,S,N,N,…,368216,Metropolitana I,5,33005,33
2,130260,Norte,AM,13,AMAZONAS,Manaus,MANAUS,S,N,S,…,297036,Entorno Manaus e Alto Rio Negro,4,13001,13
3,230440,Nordeste,CE,23,CEARA,Fortaleza,FORTALEZA,S,N,N,…,203370,1ª Região Fortaleza,5,23001,23
4,292740,Nordeste,BA,29,BAHIA,Salvador,SALVADOR,S,N,N,…,169924,Salvador,5,29020,29
5,150140,Norte,PA,15,PARA,Belém,BELEM,S,N,S,…,126918,Metropolitana I,4,15006,15
6,310620,Sudeste,MG,31,MINAS GERAIS,Belo Horizonte,BELO HORIZONTE,S,N,N,…,125370,Belo Horizonte Nova Lima Caeté,5,31008,31
7,431490,Sul,RS,43,RIO GRANDE DO SUL,Porto Alegre,PORTO ALEGRE,S,N,N,…,105321,Região 10 - Capital e Vale do Gravataí,5,43010,43
8,410690,Sul,PR,41,PARANA,Curitiba,CURITIBA,S,N,N,…,101337,2ª RS Metropolitana,5,41002,41


## Ajustando valores

### Valor Booleanos

In [11]:
df_mun[:, dt.update(capital = f.capital == 'S')]
df_mun[:, dt.update(fronteira = f.fronteira == 'S')]
df_mun[:, dt.update(amazonia = f.amazonia == 'S')]
df_mun

,cod_municipio,regiao,uf,cod_uf,nome_uf,municipio,nome_municipio,capital,fronteira,amazonia,…,count,nome_regiao_saude,socioecon,regiao_saude,prefix
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪,▪,▪,,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,355030,Sudeste,SP,35,SAO PAULO,São Paulo,SAO PAULO,1,0,0,…,806352,São Paulo,5,35016,35
1,330455,Sudeste,RJ,33,RIO DE JANEIRO,Rio de Janeiro,RIO DE JANEIRO,1,0,0,…,368216,Metropolitana I,5,33005,33
2,130260,Norte,AM,13,AMAZONAS,Manaus,MANAUS,1,0,1,…,297036,Entorno Manaus e Alto Rio Negro,4,13001,13
3,230440,Nordeste,CE,23,CEARA,Fortaleza,FORTALEZA,1,0,0,…,203370,1ª Região Fortaleza,5,23001,23
4,292740,Nordeste,BA,29,BAHIA,Salvador,SALVADOR,1,0,0,…,169924,Salvador,5,29020,29
5,150140,Norte,PA,15,PARA,Belém,BELEM,1,0,1,…,126918,Metropolitana I,4,15006,15
6,310620,Sudeste,MG,31,MINAS GERAIS,Belo Horizonte,BELO HORIZONTE,1,0,0,…,125370,Belo Horizonte Nova Lima Caeté,5,31008,31
7,431490,Sul,RS,43,RIO GRANDE DO SUL,Porto Alegre,PORTO ALEGRE,1,0,0,…,105321,Região 10 - Capital e Vale do Gravataí,5,43010,43
8,410690,Sul,PR,41,PARANA,Curitiba,CURITIBA,1,0,0,…,101337,2ª RS Metropolitana,5,41002,41


# Exportando tabela

In [12]:
output_mun = get_path('GESTANTES', 'municipios.csv.gzip')
df_mun.to_csv(output_mun)

In [13]:
dt.fread(output_mun)

,cod_municipio,regiao,uf,cod_uf,nome_uf,municipio,nome_municipio,capital,fronteira,amazonia,…,count,nome_regiao_saude,socioecon,regiao_saude,prefix
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪,▪,▪,,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,355030,Sudeste,SP,35,SAO PAULO,São Paulo,SAO PAULO,1,0,0,…,806352,São Paulo,5,35016,35
1,330455,Sudeste,RJ,33,RIO DE JANEIRO,Rio de Janeiro,RIO DE JANEIRO,1,0,0,…,368216,Metropolitana I,5,33005,33
2,130260,Norte,AM,13,AMAZONAS,Manaus,MANAUS,1,0,1,…,297036,Entorno Manaus e Alto Rio Negro,4,13001,13
3,230440,Nordeste,CE,23,CEARA,Fortaleza,FORTALEZA,1,0,0,…,203370,1ª Região Fortaleza,5,23001,23
4,292740,Nordeste,BA,29,BAHIA,Salvador,SALVADOR,1,0,0,…,169924,Salvador,5,29020,29
5,150140,Norte,PA,15,PARA,Belém,BELEM,1,0,1,…,126918,Metropolitana I,4,15006,15
6,310620,Sudeste,MG,31,MINAS GERAIS,Belo Horizonte,BELO HORIZONTE,1,0,0,…,125370,Belo Horizonte Nova Lima Caeté,5,31008,31
7,431490,Sul,RS,43,RIO GRANDE DO SUL,Porto Alegre,PORTO ALEGRE,1,0,0,…,105321,Região 10 - Capital e Vale do Gravataí,5,43010,43
8,410690,Sul,PR,41,PARANA,Curitiba,CURITIBA,1,0,0,…,101337,2ª RS Metropolitana,5,41002,41


In [14]:
df_mun.names

('cod_municipio',
 'regiao',
 'uf',
 'cod_uf',
 'nome_uf',
 'municipio',
 'nome_municipio',
 'capital',
 'fronteira',
 'amazonia',
 'macroregiao_saude',
 'microregiao_saude',
 'latitude',
 'longitude',
 'altitude',
 'area',
 'count',
 'nome_regiao_saude',
 'socioecon',
 'regiao_saude',
 'prefix')